In [1]:
# === CELDA 1: Importar librerías y configurar rutas ===
import os
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import joblib

# === Rutas ===
BASE_DIR = r"C:\Users\User\Downloads\Mini reinanse"
DATA_DIR = os.path.join(BASE_DIR, "data")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
HMM_MODELS_DIR = os.path.join(BASE_DIR, "modelos_hmm")
ESTADO_DIR = os.path.join(DATA_DIR, "estado")

os.makedirs(HMM_MODELS_DIR, exist_ok=True)
os.makedirs(ESTADO_DIR, exist_ok=True)

print("✅ Rutas configuradas correctamente")


✅ Rutas configuradas correctamente


In [2]:
# === CELDA 2: Cargar dataset maestro generado en 01B ===
file_parquet = os.path.join(PROCESSED_DIR, "dataset_master.parquet")
file_csv = os.path.join(PROCESSED_DIR, "dataset_master.csv")

if os.path.exists(file_parquet):
    df = pd.read_parquet(file_parquet)
    print("✅ Dataset cargado desde Parquet")
elif os.path.exists(file_csv):
    df = pd.read_csv(file_csv)
    print("✅ Dataset cargado desde CSV")
else:
    raise FileNotFoundError("❌ No se encontró dataset_master.parquet ni dataset_master.csv.")

df = df.sort_values(["symbol", "time"]).reset_index(drop=True)
print("✅ Filas:", df.shape[0], "| Columnas:", df.shape[1], "| Símbolos:", df['symbol'].nunique())
df.head()


✅ Dataset cargado desde Parquet
✅ Filas: 509661 | Columnas: 16 | Símbolos: 681


,time,retornos,equity,symbol,return,log_return,mom_63,drawdown,cumulative_return,MA_20,MA_50,MA_200,mom_21,vol_20,vol_63,rolling_peak
0,2022-11-18,-0.010611,146.07,A.US,-0.010611,-0.010668,-0.027932,0.000000,-0.010611,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,146.07
1,2022-11-21,-0.010611,144.52,A.US,-0.010611,-0.010668,-0.027932,-0.010611,-0.010611,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,146.07
2,2022-11-22,0.085940,156.94,A.US,0.085940,0.082446,-0.027932,0.000000,0.075328,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,156.94
3,2022-11-23,-0.010768,155.25,A.US,-0.010768,-0.010827,-0.027932,-0.010768,0.064560,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,156.94
4,2022-11-25,0.009275,156.69,A.US,0.009275,0.009233,-0.027932,-0.001593,0.073835,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,156.94


In [3]:
# === CELDA 3: Seleccionar columnas para HMM ===
FEATURES_HMM = ["return", "vol_20", "mom_21", "drawdown"]

# Verificar que existen
missing = [f for f in FEATURES_HMM if f not in df.columns]
if missing:
    raise ValueError(f"❌ Faltan columnas para HMM: {missing}")

# Eliminar filas sin datos en esas columnas
df_hmm = df.dropna(subset=FEATURES_HMM).copy()

print("✅ Dataset listo para HMM")
print("✅ Filas tras limpieza:", df_hmm.shape)


✅ Dataset listo para HMM
✅ Filas tras limpieza: (509661, 16)


In [4]:
# === CELDA 4: Función para entrenar HMM por símbolo ===

def entrenar_hmm(symbol, n_states=3):
    df_sym = df_hmm[df_hmm["symbol"] == symbol]

    if df_sym.shape[0] < 200:
        return False, "Datos insuficientes (<200 filas)"

    X = df_sym[FEATURES_HMM].values  # numpy array para hmmlearn

    model = GaussianHMM(
        n_components=n_states,
        covariance_type='diag',
        n_iter=200,
        random_state=42
    )

    model.fit(X)

    # Guardar modelo entrenado
    modelo_path = os.path.join(HMM_MODELS_DIR, f"modelo_{symbol}.pkl")
    joblib.dump(model, modelo_path)

    # Predecir estados
    estados = model.predict(X)

    # Crear DataFrame con estados
    df_estado = pd.DataFrame({
        "time": df_sym["time"].values,
        "symbol": symbol,
        "estado": estados
    })

    # Guardar estados sin clasificar
    df_estado.to_csv(os.path.join(ESTADO_DIR, f"estado_{symbol}.csv"), index=False)

    return True, "OK"


In [5]:
# === CELDA 5: Clasificar automáticamente los estados HMM ===

def clasificar_estados(symbol):
    estado_file = os.path.join(ESTADO_DIR, f"estado_{symbol}.csv")
    if not os.path.exists(estado_file):
        return False, "No hay archivo de estados"

    # Cargar estados predecidos
    df_estado = pd.read_csv(estado_file)
    df_sym = df[df["symbol"] == symbol][["time", "return"]]

    # ✅ Convertir ambas columnas time a datetime
    df_estado["time"] = pd.to_datetime(df_estado["time"], errors="coerce")
    df_sym["time"] = pd.to_datetime(df_sym["time"], errors="coerce")

    # ✅ Hacer el merge limpio
    df_merge = df_estado.merge(df_sym, on="time", how="left")

    # Calcular retorno medio por estado
    stats = df_merge.groupby("estado")["return"].mean().sort_values()

    estado_bajista = stats.index[0]
    estado_alcista = stats.index[-1]
    estado_lateral = [e for e in stats.index if e not in [estado_bajista, estado_alcista]][0]

    mapping = {
        estado_bajista: "BAJISTA",
        estado_alcista: "ALCISTA",
        estado_lateral: "LATERAL"
    }

    df_merge["estado_clasificado"] = df_merge["estado"].map(mapping)

    # Guardar
    f_out = os.path.join(ESTADO_DIR, f"estado_{symbol}_clasificado.csv")
    df_merge.to_csv(f_out, index=False)

    return True, mapping


In [6]:
# === CELDA 6: Ejecutar entrenamiento en todos los símbolos ===

errores = []
symbols = df_hmm["symbol"].unique()

for sym in symbols:
    ok, msg = entrenar_hmm(sym, n_states=3)
    if not ok:
        errores.append((sym, msg))
        print(f"❌ {sym}: {msg}")
        continue

    ok2, mapping = clasificar_estados(sym)
    if ok2:
        print(f"✅ {sym}: {mapping}")
    else:
        print(f"⚠ {sym}: Estados sin clasificar")

print("🏁 Entrenamiento completado.")
print("Errores:", errores[:10])


C:\Users\User\anaconda3\envs\mt4_trading\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
Model is not converging.  Current: 6820.3814305995875 is not greater than 6820.855155226027. Delta is -0.47372462643943436
C:\Users\User\anaconda3\envs\mt4_trading\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
Model is not converging.  Current: 5223.8386929175385 is not greater than 5223.842588592264. Delta is -0.003895674725754361


✅ A.US: {0: 'BAJISTA', 1: 'ALCISTA', 2: 'LATERAL'}
✅ AA.US: {0: 'BAJISTA', 2: 'ALCISTA', 1: 'LATERAL'}


C:\Users\User\anaconda3\envs\mt4_trading\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


✅ AAP.US: {0: 'BAJISTA', 1: 'ALCISTA', 2: 'LATERAL'}


C:\Users\User\anaconda3\envs\mt4_trading\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


...

C:\Users\User\anaconda3\envs\mt4_trading\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
Model is not converging.  Current: 5732.775977782607 is not greater than 5733.310385494851. Delta is -0.5344077122445015


✅ LMT.US: {2: 'BAJISTA', 1: 'ALCISTA', 0: 'LATERAL'}


C:\Users\User\anaconda3\envs\mt4_trading\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
Model is not converging.  Current: 8473.275333732176 is not greater than 8473.702413481245. Delta is -0.42707974906988966


✅ LNC.US: {0: 'BAJISTA', 1: 'ALCISTA', 2: 'LATERAL'}


IndexError: list index out of range